In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pip install ultralytics

# **Import The Libraries**

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import yaml
from PIL import Image
from ultralytics import YOLO
from IPython.display import Video
import numpy as np 
import pandas as pd 


In [ ]:
sns.set(rc={'axes.facecolor': '#eae8fa'}, style='darkgrid')

In [ ]:
model1 = YOLO("yolov8n.pt")  
model2 = YOLO("yolov8s.pt")
model3 = YOLO("yolov8m.pt")
model4 = YOLO("yolov8l.pt")
model5 = YOLO("yolov8x.pt")
image_path = '/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/sample_image.jpg'
results1 = model1(image_path, show=True)  
results2 = model2(image_path, show=True)  
results3 = model3(image_path, show=True)  
results4 = model4(image_path, show=True)  
results5 = model5(image_path, show=True)  

results1[0].save("output1.jpg")
results2[0].save("output2.jpg")
results3[0].save("output3.jpg")
results4[0].save("output4.jpg")
results5[0].save("output5.jpg")


In [ ]:
img1 = Image.open("output1.jpg")
img2 = Image.open("output2.jpg")
img3 = Image.open("output3.jpg")
img4 = Image.open("output4.jpg")
img5 = Image.open("output5.jpg")



fig, axes = plt.subplots(5, 1, figsize=(15,15)) 
images = [img1, img2, img3, img4, img5]

for ax, img in zip(axes, images):
    ax.imshow(img)
    ax.axis("off")  

plt.tight_layout() 
plt.show()

# **So the best model is YOLOv8n.It is working well on the image with good accuracy and precision.**

In [ ]:
model = YOLO("yolov8s.pt") 

input_video_path ="/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/sample_video.mp4"

output_video_path = "output_video.mp4"

cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps         = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")  
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.65)  # Confidence threshold = 0.65

    annotated_frame = results[0].plot()

    out.write(annotated_frame)

cap.release()
out.release()

print(f"Processed video saved as {output_video_path}")


In [ ]:
model6=YOLO('yolov8n.pt')
image_path = '/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/sample_image.jpg'

results = model6.predict(source=image_path, imgsz=640, conf=0.5)   # Confidence threshold: 50% 

sample_image = results[0].plot(line_width=2)
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,15))
plt.imshow(sample_image)
plt.title('Detected Objects in Sample Image by the Pre-trained YOLOv8 Model on COCO Dataset', fontsize=15)
plt.axis('off')
plt.show()

# **Dataset Exploration**

In [ ]:
dataset_path=('/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset')
yaml_file_path=os.path.join(dataset_path,'data.yaml')
with open(yaml_file_path,'r') as file:
    yaml_content=yaml.load(file,Loader=yaml.FullLoader)
    print(yaml.dump(yaml_content, default_flow_style=True))

In [ ]:
train_image_path=os.path.join(dataset_path,'train','images')
valid_image_path=os.path.join(dataset_path,'valid','images')

num_train_image=0
num_valid_image=0

train_image_sizes = set()
valid_image_sizes = set()

for filename in os.listdir(train_image_path):
    if filename.endswith('.jpg'):
        num_train_image+=1
        image_path=os.path.join(train_image_path,filename)
    with Image.open(image_path) as img:
        train_image_sizes.add(img.size)

for filename in os.listdir(valid_image_path):
    if filename.endswith('.jpg'):
        num_valid_image+=1
        image_path=os.path.join(valid_image_path,filename)
    with Image.open(image_path) as img:
        valid_image_sizes.add(img.size)


print(f'Number of Training Images is {num_train_image}')
print(f'Number of Validation Images is {num_valid_image}')

if len(train_image_sizes)==1:
    print(f'All Traing Image Has Same Size:{train_image_sizes.pop()}')
else:
    print('All Images are varying images sizes')

if len(valid_image_sizes)==1:
    print(f'All Validation Image Has Same Size:{valid_image_sizes.pop()}')
else:
    print('All Images are varying images sizes')

In [ ]:
image_files=[file for file in os.listdir(train_image_path) if file.endswith('.jpg')]
selected_images=[image_files[i] for i in range(0,len(image_files),len(image_files) // 12)]

fig,axes=plt.subplots(3,4,figsize=(20,16))
for ax,img_file in zip(axes.ravel(),selected_images):
    image_path=os.path.join(train_image_path,img_file)
    image=Image.open(image_path)
    ax.imshow(image)
    ax.axis('off')

plt.suptitle('Sample image from training set',fontsize=30)
plt.tight_layout()
# plt.show()

# **Fine Model Tuning** 


In [ ]:
pip install tensorboard


In [ ]:
result=model.train(
    data=yaml_file_path,
    epochs=100,
    imgsz=640,
    device=0,
    patience=0,
    batch=32,
    optimizer='auto',
    lr0=0.001,
    lrf=0.1,
    dropout=0.1,
    seed=0,
    project="runs/train",
    name="exp"
    
)

In [ ]:
exp_path='/kaggle/working/runs/train/exp'
image_files=[file for file in os.listdir(exp_path) if file.endswith('.png')]
# selected_images=[image_files[i] for i in range(0,len(image_files),len(image_files) // 12)]

fig,axes=plt.subplots(4,2,figsize=(40,40))
for ax,img_file in zip(axes.ravel(),image_files):
    image_path=os.path.join(exp_path,img_file)
    image=Image.open(image_path)
    ax.imshow(image)
    ax.axis('off')

plt.suptitle('Parameter Curve After Model Tuning',fontsize=10)
plt.tight_layout()
# plt.show()

In [ ]:
exp_path='/kaggle/working/runs/train/exp'
image_files=[file for file in os.listdir(exp_path) if file.endswith('.jpg')]
# selected_images=[image_files[i] for i in range(0,len(image_files),len(image_files) // 12)]
fig,axes=plt.subplots(4,3,figsize=(40,40))
for ax,img_file in zip(axes.ravel(),image_files):
    image_path=os.path.join(exp_path,img_file)
    image=Image.open(image_path)
    ax.imshow(image)
    ax.axis('off')

plt.suptitle('Vehile Detection After Model Tuning',fontsize=10)
plt.tight_layout()
plt.show()

# **Learning Curve Analysis**

In [ ]:
def plot_leqarning_curve(df,train_loss_col,valid_loss_col,title):
    plt.figure(figsize=(20,12))
    sns.lineplot(data=df,x='epoch',y=train_loss_col,label='Train Loss',color='blue',linestyle='-',linewidth=2)
    sns.lineplot(data=df,x='epoch',y=valid_loss_col,label='Valid Loss',color='Orange',linestyle='--',linewidth=2)
    plt.legend()
    plt.title(title)
    plt.xlabel('epoch')
    plt.ylabel('Loss')
    plt.show()
    


In [ ]:
results_csv_path=os.path.join(exp_path,'results.csv')
df=pd.read_csv(results_csv_path)
df.columns=df.columns.str.strip()
plot_leqarning_curve(df,'train/box_loss','val/box_loss','Box Loss Learning Curve')
plot_leqarning_curve(df,'train/cls_loss','val/cls_loss','Classification Loss Learning Curve')
plot_leqarning_curve(df,'train/dfl_loss','val/dfl_loss','Distribution Focal Loss Learning Curve')


# **Confusion Matrix Evaluation**

In [ ]:
confusion_matric_path=os.path.join(exp_path,'confusion_matrix_normalized.png')
cm_img=cv2.imread(confusion_matric_path)
cm_img=cv2.cvtColor(cm_img,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20,20),dpi=400)
plt.imshow(cm_img)
plt.title('Normalizes Confusion Matrix',fontsize=30)
plt.tight_layout()
plt.axis('off')
plt.show()

# **Performance Metrix Assessment**

In [ ]:
best_model_path=os.path.join(exp_path,'weights/best.pt')
best_model=YOLO(best_model_path)
metrics=best_model.val(split='val')

In [ ]:
metrics_df=pd.DataFrame.from_dict(metrics.results_dict,orient='index',columns=['Metric Value'])
metrics_df.round(3)

# **Inference On Validation Set Images**

In [ ]:
valid_images_path=os.path.join(dataset_path,'valid','images')
image_file=[file for file in os.listdir(valid_images_path) if file.endswith('.jpg')]
selected_image=[image_file[i] for i in range(0,len(image_file),len(image_file) // 12)]

fig,axes=plt.subplots(4,3,figsize=(40,40))
fig.suptitle('Valid Inference Set',fontsize=40)

for i,ax in enumerate(axes.flatten()):
    image_path = os.path.join(valid_images_path, selected_image[i])
    results = best_model.predict(source=image_path, imgsz=640, conf=0.5)
    annotated_image = results[0].plot(line_width=1)
    annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    ax.imshow(annotated_image_rgb)
    ax.axis('off')

plt.tight_layout()
plt.show()

# **Inference On Unseen Sample Image**

In [ ]:
sample_image_path = '/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/sample_image.jpg'

results = best_model.predict(source=sample_image_path, imgsz=640, conf=0.7)   # Confidence threshold: 70% 

sample_image = results[0].plot(line_width=2)
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,15))
plt.imshow(sample_image)
plt.title('Detected Objects in Sample Image by the Post-trained Best Model on COCO Dataset', fontsize=15)
plt.axis('off')
plt.show()

# **Inference On Unseen Sample Video**

In [ ]:
dataset_video_path = '/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset/sample_video.mp4'
video_path = '/kaggle/working/bsample_video.mp4'
shutil.copyfile(dataset_video_path, video_path)
best_model.predict(source=video_path, save=True)

In [ ]:
# Convert the .avi video generated by the YOLOv8 prediction to .mp4 format for compatibility with notebook display
!ffmpeg -y -loglevel panic -i /kaggle/working/runs/detect/predict/best_sample_video.avi processed_sample_video.mp4

Video("processed_sample_video.mp4", embed=True, width=960)

# **Real-Time Traffic Density Estimation With YOLO**

In [ ]:
# Define the threshold for considering traffic as heavy
heavy_traffic_threshold = 10

# Define the vertices for the quadrilaterals
vertices1 = np.array([(465, 350), (609, 350), (510, 630), (2, 630)], dtype=np.int32)
vertices2 = np.array([(678, 350), (815, 350), (1203, 630), (743, 630)], dtype=np.int32)

# Define the vertical range for the slice and lane threshold
x1, x2 = 325, 635 
lane_threshold = 609

# Define the positions for the text annotations on the image
text_position_left_lane = (10, 50)
text_position_right_lane = (820, 50)
intensity_position_left_lane = (10, 100)
intensity_position_right_lane = (820, 100)

# Define font, scale, and colors for the annotations
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_color = (255, 255, 255)    # White color for text
background_color = (0,255,0)  # Green background for text
        
# Open the video
cap = cv2.VideoCapture('best_sample_video.mp4')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('traffic_density_analysis.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Read until video is completed
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Create a copy of the original frame to modify
        detection_frame = frame.copy()
    
        # Black out the regions outside the specified vertical range
        detection_frame[:x1, :] = 0  # Black out from top to x1
        detection_frame[x2:, :] = 0  # Black out from x2 to the bottom of the frame
        
        # Perform inference on the modified frame
        results = best_model.predict(detection_frame, imgsz=640, conf=0.4)
        processed_frame = results[0].plot(line_width=1)
        
        # Restore the original top and bottom parts of the frame
        processed_frame[:x1, :] = frame[:x1, :].copy()
        processed_frame[x2:, :] = frame[x2:, :].copy()        
        
        # Draw the quadrilaterals on the processed frame
        cv2.polylines(processed_frame, [vertices1], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.polylines(processed_frame, [vertices2], isClosed=True, color=(255, 0, 0), thickness=2)
        
        # Retrieve the bounding boxes from the results
        bounding_boxes = results[0].boxes

        # Initialize counters for vehicles in each lane
        vehicles_in_left_lane = 0
        vehicles_in_right_lane = 0

        # Loop through each bounding box to count vehicles in each lane
        for box in bounding_boxes.xyxy:
            # Check if the vehicle is in the left lane based on the x-coordinate of the bounding box
            if box[0] < lane_threshold:
                vehicles_in_left_lane += 1
            else:
                vehicles_in_right_lane += 1
                
        # Determine the traffic intensity for the left lane
        traffic_intensity_left = "Heavy" if vehicles_in_left_lane > heavy_traffic_threshold else "Smooth"
        # Determine the traffic intensity for the right lane
        traffic_intensity_right = "Heavy" if vehicles_in_right_lane > heavy_traffic_threshold else "Smooth"


        # Add a background rectangle for the left lane vehicle count
        cv2.rectangle(processed_frame, (text_position_left_lane[0]-10, text_position_left_lane[1] - 25), 
                      (text_position_left_lane[0] + 460, text_position_left_lane[1] + 10), background_color, -1)

        # Add the vehicle count text on top of the rectangle for the left lane
        cv2.putText(processed_frame, f'Vehicles in Left Lane: {vehicles_in_left_lane}', text_position_left_lane, 
                    font, font_scale, font_color, 2, cv2.LINE_AA)

        # Add a background rectangle for the left lane traffic intensity
        cv2.rectangle(processed_frame, (intensity_position_left_lane[0]-10, intensity_position_left_lane[1] - 25), 
                      (intensity_position_left_lane[0] + 460, intensity_position_left_lane[1] + 10), background_color, -1)

        # Add the traffic intensity text on top of the rectangle for the left lane
        cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity_left}', intensity_position_left_lane, 
                    font, font_scale, font_color, 2, cv2.LINE_AA)

        # Add a background rectangle for the right lane vehicle count
        cv2.rectangle(processed_frame, (text_position_right_lane[0]-10, text_position_right_lane[1] - 25), 
                      (text_position_right_lane[0] + 460, text_position_right_lane[1] + 10), background_color, -1)

        # Add the vehicle count text on top of the rectangle for the right lane
        cv2.putText(processed_frame, f'Vehicles in Right Lane: {vehicles_in_right_lane}', text_position_right_lane, 
                    font, font_scale, font_color, 2, cv2.LINE_AA)

        # Add a background rectangle for the right lane traffic intensity
        cv2.rectangle(processed_frame, (intensity_position_right_lane[0]-10, intensity_position_right_lane[1] - 25), 
                      (intensity_position_right_lane[0] + 460, intensity_position_right_lane[1] + 10), background_color, -1)

        # Add the traffic intensity text on top of the rectangle for the right lane
        cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity_right}', intensity_position_right_lane, 
                    font, font_scale, font_color, 2, cv2.LINE_AA)

        # Write the processed frame to the output video
        out.write(processed_frame)
        
        # Uncomment the following 3 lines if running this code on a local machine to view the real-time processing results
        cv2.imshow('Real-time Analysis', processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press Q on keyboard to exit the loop
            break
    else:
        break

# Release the video capture and video write objects
cap.release()
out.release()

# Close all the frames
# cv2.destroyAllWindows()

In [ ]:
!ffmpeg -y -loglevel panic -i /kaggle/working/traffic_density_analysis.avi traffic_density_analysis.mp4

Video("traffic_density_analysis.mp4", embed=True, width=960)

In [ ]:
best_model.export(format='onnx')
